# 資料驗證

進行資料分析前，我們通常會預期資料已經是在可分析的狀態 (ready for analysis)，可減少資料清理的時間。因此取得資料後，若能夠進行資料驗證，確認其格式、欄位資訊、欄位關聯是正常的之後，再進行分析，可避免一些意料之外的錯誤，導致分析結果錯誤。

在這份教學中，我們會使用 `patito` 這個套件進行資料驗證。它是基於 Python 上非常有名的格式驗證套件 `pydantic` 與先前介紹過的資料集套件 `polars`。原則上，除了內建的方法外，只要你可以用 `polars` 表達式 (expression) 寫出的規則，都可以進行驗證，是一個非常強大的套件。

接下來就讓我來看看如何使用 `patito` 進行資料驗證！

首先，我們需要匯入此專案中需要的套件：

* `polars`: 資料處理
* `patito`: 資料驗證
* `typing`: 格式指定

在下方程式碼中，`as` 是用來幫套件取別稱。如此一來，在接下來的程式碼中，若需要用到套件時，就不需要輸入全名，例如需要用到 `polars` 時，只需要輸入 `pl` 就可以了。

In [1]:
import polars as pl
import patito as pt
from typing import Literal, Optional, Dict, List
import re

下方是一段簡短的函式，幫助我們閱讀 `patito` 告訴我們的驗證結果。

In [2]:
class ErrorReporter:
    def __init__(self, validator_class, df: pl.DataFrame):
        self.validator = validator_class
        self.df = df
        self.error_report = self._val()

    def _val(self):
        try:
            self.validator.validate(self.df)
            return 'Pass'
        except Exception as e:
            return self._get_error_report(e.errors(), self.df.columns)
    
    def _get_error_report(self, error_list: List[Dict], col_list: List[str]) -> pl.DataFrame:
        """
        Parse a list of error items into a DataFrame with columns: column, pass, failed_count, msg, type
        
        Args:
            error_list (List[Dict]): List of error items
            col_list (List[str]): List of column names
            
        Returns:
            pl.DataFrame: DataFrame with parsed error information
        """
        column_list = []
        failed_count_list = []
        msg_list = []
        type_list = []
        
        for error in error_list:
            # Extract column name from loc tuple
            column = error['loc'][0]
            
            # Extract failed count if present in msg
            failed_count = None
            msg = error['msg']
            count_match = re.match(r'^\d+', msg)
            if count_match:
                failed_count = int(count_match.group(0))
                
            column_list.append(column)
            failed_count_list.append(failed_count)
            msg_list.append(msg)
            type_list.append(error['type'])
        
        # Create DataFrame and order columns
        df_error = pl.DataFrame({
            'column': column_list,
            'pass': False,
            'failed_count': failed_count_list,
            'msg': msg_list,
            'type': type_list
        })
        
        df_error = df_error.join(pl.DataFrame({'column': col_list}), on='column', how='full')\
                        .with_columns(
                            pl.when(pl.col('pass').is_not_null())
                                    .then(pl.col('pass'))
                                    .otherwise(True).alias('pass'),
                            pl.when(pl.col('column').is_not_null())
                                    .then(pl.col('column'))
                                    .otherwise(pl.col('column_right')).alias('column'),
                            pl.when(pl.col('failed_count').is_not_null())
                                    .then(pl.col('failed_count'))
                                    .otherwise(0).alias('failed_count')
                                        )\
                        .drop('column_right')
        
        return df_error
    
    def report(self) -> pl.DataFrame:
        return self.error_report
    
    def summary(self) -> pl.DataFrame:
        if type(self.error_report) == 'str':
            return self.error_report
        
        return self.error_report.group_by('column').agg([
            pl.col('pass').all().alias('pass')
        ])

In [3]:
import re

def parse_error_items(error_list, col):
    """
    Parse a list of error items into a DataFrame with columns: column, pass, failed_count, msg, type
    
    Args:
        error_list (list): List of dictionaries containing error information
        col (list): List of column names
        
    Returns:
        pl.DataFrame: DataFrame with parsed error information
    """
    col_list = []
    failed_count_list = []
    msg_list = []
    type_list = []
    
    for error in error_list:
        # Extract column name from loc tuple
        column = error['loc'][0]
        
        # Extract failed count if present in msg
        failed_count = None
        msg = error['msg']
        count_match = re.match(r'^\d+', msg)
        if count_match:
            failed_count = int(count_match.group(0))
            
        col_list.append(column)
        failed_count_list.append(failed_count)
        msg_list.append(msg)
        type_list.append(error['type'])
    
    # Create DataFrame and order columns
    df_error = pl.DataFrame({
        'column': col_list,
        'pass': False,
        'failed_count': failed_count_list,
        'msg': msg_list,
        'type': type_list
    })
    
    df_error = df_error.join(pl.DataFrame({'column': col}), on='column', how='full')\
                    .with_columns(
                        pl.when(pl.col('pass').is_not_null())
                                .then(pl.col('pass'))
                                .otherwise(True).alias('pass'),
                        pl.when(pl.col('column').is_not_null())
                                .then(pl.col('column'))
                                .otherwise(pl.col('column_right')).alias('column'),
                        pl.when(pl.col('failed_count').is_not_null())
                                .then(pl.col('failed_count'))
                                .otherwise(0).alias('failed_count')
                                    )\
                    .drop('column_right')
    
    return df_error

def val(validator_class, data):
    try:
        validator_class.validate(data)
        print('Pass')
    except Exception as e:
        return parse_error_items(e.errors(), data.columns)

匯入欲進行驗證的資料。

In [4]:
df = pl.read_csv('Tutorial_Data/penguins_v.csv')

In [5]:
df.head()

ID,Species,Island,Clutch_Completion,Culmen_Length,Culmen_Depth,Flipper_Length,Body_Mass,Sex,Comments,Island_Code
i64,str,str,str,f64,f64,i64,i64,str,str,i64
1,"""Adelie Penguin (Pygoscelis ade…","""Torgersen""","""Yes""",39.1,18.7,181,3750,"""MALE""","""Not enough blood for isotopes.""",0
2,"""Adelie Penguin (Pygoscelis ade…","""Torgersen""","""Yes""",39.5,17.4,186,3800,"""FEMALE""",null,0
3,"""Adelie Penguin (Pygoscelis ade…","""Torgersen""","""Yes""",40.3,18.0,195,3250,"""FEMALE""",null,0
4,"""Adelie Penguin (Pygoscelis ade…","""Torgersen""","""Yes""",null,null,null,null,null,"""Adult not sampled.""",0
5,"""Adelie Penguin (Pygoscelis ade…","""Torgersen""","""Yes""",36.7,19.3,193,3450,"""FEMALE""",null,0


In [6]:
df.describe()

statistic,ID,Species,Island,Clutch_Completion,Culmen_Length,Culmen_Depth,Flipper_Length,Body_Mass,Sex,Comments,Island_Code
str,f64,str,str,str,f64,f64,f64,f64,str,str,f64
"""count""",344.0,"""344""","""344""","""344""",342.0,342.0,342.0,342.0,"""334""","""26""",344.0
"""null_count""",0.0,"""0""","""0""","""0""",2.0,2.0,2.0,2.0,"""10""","""318""",0.0
"""mean""",63.151163,null,null,null,68.83655,17.15117,200.915205,4201.754386,null,null,1.206395
"""std""",40.430199,null,null,null,328.085458,1.974793,14.061714,801.954536,null,null,0.687977
"""min""",1.0,"""Adelie Penguin (Pygoscelis ade…","""Biscoe""","""No""",4.1,13.1,172.0,2700.0,""".""","""Adult not sampled.""",0.0
"""25%""",29.0,null,null,null,39.3,15.6,190.0,3550.0,null,null,1.0
"""50%""",58.0,null,null,null,44.5,17.3,197.0,4050.0,null,null,1.0
"""75%""",95.0,null,null,null,48.6,18.7,213.0,4750.0,null,null,2.0
"""max""",152.0,"""Gentoo penguin (Pygoscelis pap…","""Torgersen""","""Yes""",4730.0,21.5,231.0,6300.0,"""MALE""","""Sexing primers did not amplify…",2.0


# 0. 欄位檢查

檢查資料集的欄位數目及名稱是否正確。

In [7]:
# 當實際欄位較多時

class P00(pt.Model):
    ID: int
    Species: str
    Island: str
    Clutch_Completion: str
    Culmen_Length: float
    Culmen_Depth: float
    Flipper_Length: float
    Body_Mass: float
    Sex: str
    Comments: str

In [8]:
val(P00, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",true,0,null,null
"""Species""",true,0,null,null
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Body_Mass""",false,2,"""2 missing values""","""value_error.missingvalues"""
"""Body_Mass""",false,0,"""Polars dtype Int64 does not ma…","""type_error.columndtype"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""


In [9]:
v = ErrorReporter(P00, df)

In [10]:
v.summary()

column,pass
str,bool
"""Culmen_Depth""",false
"""Clutch_Completion""",true
"""Comments""",false
"""Sex""",false
"""Island_Code""",false
…,…
"""Culmen_Length""",false
"""ID""",true
"""Island""",true


In [11]:
# 當實際欄位較少時

val(P00, df.select(['Species', 'Island', 'Clutch_Completion', 'Culmen_Length', 'Culmen_Depth', 'Flipper_Length']))

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""Body_Mass""",false,0,"""Missing column""","""type_error.missingcolumns"""
"""ID""",false,0,"""Missing column""","""type_error.missingcolumns"""
"""Sex""",false,0,"""Missing column""","""type_error.missingcolumns"""
"""Comments""",false,0,"""Missing column""","""type_error.missingcolumns"""
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Flipper_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
"""Flipper_Length""",false,0,"""Polars dtype Int64 does not ma…","""type_error.columndtype"""
"""Species""",true,0,null,null


# 1. 資料型態

確認每個欄位的資料型態皆正確。

In [12]:
class P01(pt.Model):
    ID: int
    Species: str
    Island: str
    Clutch_Completion: str
    Culmen_Length: float
    Culmen_Depth: float
    Flipper_Length: float
    Body_Mass: float
    Sex: str
    Comments: str
    Island_Code: int

In [13]:
# Flipper_Length 與 Body_Mass 型態不符

val(P01, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",true,0,null,null
"""Species""",true,0,null,null
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Body_Mass""",false,2,"""2 missing values""","""value_error.missingvalues"""
"""Body_Mass""",false,0,"""Polars dtype Int64 does not ma…","""type_error.columndtype"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""


In [14]:
class P01_1(pt.Model):
    ID: int
    Species: str
    Island: str
    Clutch_Completion: str
    Culmen_Length: float
    Culmen_Depth: float
    Flipper_Length: int
    Body_Mass: int
    Sex: str
    Comments: str
    Island_Code: int

In [15]:
val(P01_1, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",true,0,null,null
"""Species""",true,0,null,null
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Flipper_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
"""Body_Mass""",false,2,"""2 missing values""","""value_error.missingvalues"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""


In [16]:
# 可使用 polars 的 dtype 來指定型態

class P01_2(pt.Model):
    ID: int
    Species: str
    Island: str
    Clutch_Completion: str
    Culmen_Length: float
    Culmen_Depth: float
    Flipper_Length: int
    Body_Mass: int = pt.Field(dtype=pl.UInt64)
    Sex: str
    Comments: str
    Island_Code: int

In [17]:
val(P01_2, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",true,0,null,null
"""Species""",true,0,null,null
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Body_Mass""",false,2,"""2 missing values""","""value_error.missingvalues"""
"""Body_Mass""",false,0,"""Polars dtype Int64 does not ma…","""type_error.columndtype"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""


# 2. 是否為可接受的值

驗證一個類別欄位的值皆為特定的幾個類別。

In [18]:
# 使用 Literal 限制欄位值，欄位的值必須是 Literal 內的其中一個

class P02(pt.Model):
    ID: int
    Species: Literal['Gentoo penguin (Pygoscelis papua)', 
                     'Adelie Penguin (Pygoscelis adeliae)', 
                     'Chinstrap penguin (Pygoscelis antarctica)']
    Island: Literal['Torgersen', 'Biscoe', 'Dream']
    Clutch_Completion: Literal['Yes', 'No']
    Culmen_Length: float
    Culmen_Depth: float
    Flipper_Length: int
    Body_Mass: int
    Sex: Literal['MALE', 'FEMALE']
    Comments: str
    Island_Code: int

In [19]:
val(P02, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",true,0,null,null
"""Species""",true,0,null,null
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Body_Mass""",false,2,"""2 missing values""","""value_error.missingvalues"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""
"""Sex""",false,0,"""Rows with invalid values: {Non…","""value_error.rowvalue"""


In [20]:
# 在此給予錯誤的 Species 類別

class P02_1(pt.Model):
    ID: int
    Species: Literal['Gentoo penguin', 
                     'Adelie Penguin', 
                     'Chinstrap penguin']
    Island: Literal['Torgersen', 'Biscoe', 'Dream']
    Clutch_Completion: Literal['Yes', 'No']
    Culmen_Length: float
    Culmen_Depth: float
    Flipper_Length: int
    Body_Mass: int
    Sex: Literal['MALE', 'FEMALE']
    Comments: str
    Island_Code: int

In [21]:
val(P02_1, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",true,0,null,null
"""Species""",false,0,"""Rows with invalid values: {'Ge…","""value_error.rowvalue"""
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Body_Mass""",false,2,"""2 missing values""","""value_error.missingvalues"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""
"""Sex""",false,0,"""Rows with invalid values: {Non…","""value_error.rowvalue"""


# 3. 是否允許空值

確認該欄位是否接受空值，並檢查欄位是否有空值。若欄位不接受空值但該欄位存在空值，則會驗證失敗。

In [22]:
# 利用 Optional 來指定欄位可以為空

class P03(pt.Model):
    ID: int
    Species: Literal['Gentoo penguin (Pygoscelis papua)', 
                     'Adelie Penguin (Pygoscelis adeliae)', 
                     'Chinstrap penguin (Pygoscelis antarctica)']
    Island: Literal['Torgersen', 'Biscoe', 'Dream']
    Clutch_Completion: Literal['Yes', 'No']
    Culmen_Length: float
    Culmen_Depth: float
    Flipper_Length: int
    Body_Mass: int
    Sex: Literal['MALE', 'FEMALE']
    Comments: Optional[str]
    Island_Code: int

In [23]:
val(P03, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",true,0,null,null
"""Species""",true,0,null,null
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Body_Mass""",false,2,"""2 missing values""","""value_error.missingvalues"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""
"""Sex""",false,0,"""Rows with invalid values: {Non…","""value_error.rowvalue"""


# 4. 是否不允許重複

確認該欄位是否接受重複值，並檢查欄位是否有重複值。若欄位不接受重複值但該欄位有值重複，則會驗證失敗。

In [24]:
# 利用 unique 來指定欄位值必須唯一

class P04(pt.Model):
    ID: int = pt.Field(unique=True)
    Species: Literal['Gentoo penguin (Pygoscelis papua)', 
                     'Adelie Penguin (Pygoscelis adeliae)', 
                     'Chinstrap penguin (Pygoscelis antarctica)']
    Island: Literal['Torgersen', 'Biscoe', 'Dream']
    Clutch_Completion: Literal['Yes', 'No']
    Culmen_Length: float
    Culmen_Depth: float
    Flipper_Length: int
    Body_Mass: int
    Sex: Literal['MALE', 'FEMALE']
    Comments: Optional[str]
    Island_Code: int

In [25]:
val(P04, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",false,316,"""316 rows with duplicated value…","""value_error.rowvalue"""
"""Species""",true,0,null,null
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Body_Mass""",false,2,"""2 missing values""","""value_error.missingvalues"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""
"""Sex""",false,0,"""Rows with invalid values: {Non…","""value_error.rowvalue"""


# 5. 字串長度

確認字串長度符合規定。

In [26]:
df.select(pl.col('Comments').str.len_chars().min().alias('Comments Min Length'),
          pl.col('Comments').str.len_chars().max().alias('Comments Max Length'))

Comments Min Length,Comments Max Length
u32,u32
18,68


In [27]:
df.select(pl.col('Comments').str.len_chars().value_counts()).unnest('Comments')

Comments,count
u32,u32
62,1
30,6
25,2
36,2
null,318
37,13
18,1
68,1


In [28]:
class P05(pt.Model):
    ID: int = pt.Field(unique=True)
    Species: Literal['Gentoo penguin (Pygoscelis papua)', 
                     'Adelie Penguin (Pygoscelis adeliae)', 
                     'Chinstrap penguin (Pygoscelis antarctica)']
    Island: Literal['Torgersen', 'Biscoe', 'Dream']
    Clutch_Completion: Literal['Yes', 'No']
    Culmen_Length: float
    Culmen_Depth: float
    Flipper_Length: int
    Body_Mass: int
    Sex: Literal['MALE', 'FEMALE']
    Comments: Optional[str] = pt.Field(max_length=50)
    Island_Code: int

In [29]:
val(P05, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",false,316,"""316 rows with duplicated value…","""value_error.rowvalue"""
"""Species""",true,0,null,null
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Body_Mass""",false,2,"""2 missing values""","""value_error.missingvalues"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""
"""Sex""",false,0,"""Rows with invalid values: {Non…","""value_error.rowvalue"""


In [30]:
class P05_1(pt.Model):
    ID: int = pt.Field(unique=True)
    Species: Literal['Gentoo penguin (Pygoscelis papua)', 
                     'Adelie Penguin (Pygoscelis adeliae)', 
                     'Chinstrap penguin (Pygoscelis antarctica)']
    Island: Literal['Torgersen', 'Biscoe', 'Dream']
    Clutch_Completion: Literal['Yes', 'No']
    Culmen_Length: float
    Culmen_Depth: float
    Flipper_Length: int
    Body_Mass: int
    Sex: Literal['MALE', 'FEMALE']
    Comments: Optional[str] = pt.Field(min_length=20)
    Island_Code: int

In [31]:
val(P05_1, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",false,316,"""316 rows with duplicated value…","""value_error.rowvalue"""
"""Species""",true,0,null,null
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Body_Mass""",false,2,"""2 missing values""","""value_error.missingvalues"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""
"""Sex""",false,0,"""Rows with invalid values: {Non…","""value_error.rowvalue"""


In [32]:
class P05_2(pt.Model):
    ID: int = pt.Field(unique=True)
    Species: Literal['Gentoo penguin (Pygoscelis papua)', 
                     'Adelie Penguin (Pygoscelis adeliae)', 
                     'Chinstrap penguin (Pygoscelis antarctica)']
    Island: Literal['Torgersen', 'Biscoe', 'Dream']
    Clutch_Completion: Literal['Yes', 'No']
    Culmen_Length: float
    Culmen_Depth: float
    Flipper_Length: int
    Body_Mass: int
    Sex: Literal['MALE', 'FEMALE']
    Comments: Optional[str] = pt.Field(min_length=20, max_length=50)
    Island_Code: int

In [33]:
val(P05_2, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",false,316,"""316 rows with duplicated value…","""value_error.rowvalue"""
"""Species""",true,0,null,null
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Body_Mass""",false,2,"""2 missing values""","""value_error.missingvalues"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""
"""Sex""",false,0,"""Rows with invalid values: {Non…","""value_error.rowvalue"""


# 6. 字串規則

利用 `re` 正則表達式來檢查字串是否符合特定格式。

In [34]:
df.filter(pl.col('Comments').is_not_null()).select(pl.col('Comments'))

Comments
str
"""Not enough blood for isotopes."""
"""Adult not sampled."""
"""Nest never observed with full …"
"""Nest never observed with full …"
"""No blood sample obtained."""
…
"""Nest never observed with full …"
"""Nest never observed with full …"
"""Nest never observed with full …"


In [35]:
df.filter(pl.col('Comments').is_not_null()).filter(~pl.col('Comments').str.starts_with('Nest')).select(pl.col('Comments'))

Comments
str
"""Not enough blood for isotopes."""
"""Adult not sampled."""
"""No blood sample obtained."""
"""No blood sample obtained for s…"
"""No blood sample obtained for s…"
…
"""Not enough blood for isotopes."""
"""Not enough blood for isotopes."""
"""Not enough blood for isotopes."""


In [36]:
# 使用 pattern 來進行 re 格式的驗證 
# 官網提到使用 regex 參數進行設定，但目前並不支援

class P06(pt.Model):
    ID: int = pt.Field(unique=True)
    Species: Literal['Gentoo penguin (Pygoscelis papua)', 
                     'Adelie Penguin (Pygoscelis adeliae)', 
                     'Chinstrap penguin (Pygoscelis antarctica)']
    Island: Literal['Torgersen', 'Biscoe', 'Dream']
    Clutch_Completion: Literal['Yes', 'No']
    Culmen_Length: float
    Culmen_Depth: float
    Flipper_Length: int
    Body_Mass: int
    Sex: Literal['MALE', 'FEMALE']
    Comments: Optional[str] = pt.Field(pattern=r'^Nest[A-Za-z0-9\s]+')
    Island_Code: int

In [37]:
val(P06, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",false,316,"""316 rows with duplicated value…","""value_error.rowvalue"""
"""Species""",true,0,null,null
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Body_Mass""",false,2,"""2 missing values""","""value_error.missingvalues"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""
"""Sex""",false,0,"""Rows with invalid values: {Non…","""value_error.rowvalue"""


# 7. 值的範圍

確認一欄位的值皆大於（等於）或/及小於（等於）特定值。

* `ge`: 大於等於
* `le`: 小於等於
* `gt`: 大於
* `lt`: 小於

In [38]:
df.select('Body_Mass').describe()

statistic,Body_Mass
str,f64
"""count""",342.0
"""null_count""",2.0
"""mean""",4201.754386
"""std""",801.954536
"""min""",2700.0
"""25%""",3550.0
"""50%""",4050.0
"""75%""",4750.0
"""max""",6300.0


In [39]:
class P07(pt.Model):
    ID: int = pt.Field(unique=True)
    Species: Literal['Gentoo penguin (Pygoscelis papua)', 
                     'Adelie Penguin (Pygoscelis adeliae)', 
                     'Chinstrap penguin (Pygoscelis antarctica)']
    Island: Literal['Torgersen', 'Biscoe', 'Dream']
    Clutch_Completion: Literal['Yes', 'No']
    Culmen_Length: float
    Culmen_Depth: float
    Flipper_Length: int
    Body_Mass: int = pt.Field(ge=2700, le=6300)
    Sex: Literal['MALE', 'FEMALE']
    Comments: Optional[str]
    Island_Code: int

In [40]:
val(P07, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",false,316,"""316 rows with duplicated value…","""value_error.rowvalue"""
"""Species""",true,0,null,null
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Body_Mass""",false,2,"""2 missing values""","""value_error.missingvalues"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""
"""Sex""",false,0,"""Rows with invalid values: {Non…","""value_error.rowvalue"""


In [41]:
class P07_1(pt.Model):
    ID: int = pt.Field(unique=True)
    Species: Literal['Gentoo penguin (Pygoscelis papua)', 
                     'Adelie Penguin (Pygoscelis adeliae)', 
                     'Chinstrap penguin (Pygoscelis antarctica)']
    Island: Literal['Torgersen', 'Biscoe', 'Dream']
    Clutch_Completion: Literal['Yes', 'No']
    Culmen_Length: float
    Culmen_Depth: float
    Flipper_Length: int
    Body_Mass: int = pt.Field(gt=2700, lt=6300)
    Sex: Literal['MALE', 'FEMALE']
    Comments: Optional[str]
    Island_Code: int

In [42]:
val(P07_1, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",false,316,"""316 rows with duplicated value…","""value_error.rowvalue"""
"""Species""",true,0,null,null
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Body_Mass""",false,2,"""2 rows with out of bound value…","""value_error.rowvalue"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""
"""Sex""",false,0,"""Rows with invalid values: {Non…","""value_error.rowvalue"""


# 8. 值為某個倍數

確認該欄位的值皆為某個數值的倍數。

In [43]:
# 確認 Body_Mass 的值是否皆為 100 的倍數

class P08(pt.Model):
    ID: int = pt.Field(unique=True)
    Species: Literal['Gentoo penguin (Pygoscelis papua)', 
                     'Adelie Penguin (Pygoscelis adeliae)', 
                     'Chinstrap penguin (Pygoscelis antarctica)']
    Island: Literal['Torgersen', 'Biscoe', 'Dream']
    Clutch_Completion: Literal['Yes', 'No']
    Culmen_Length: float
    Culmen_Depth: float
    Flipper_Length: int
    Body_Mass: int = pt.Field(multiple_of=100)
    Sex: Literal['MALE', 'FEMALE']
    Comments: Optional[str]
    Island_Code: int

In [44]:
val(P08, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",false,316,"""316 rows with duplicated value…","""value_error.rowvalue"""
"""Species""",true,0,null,null
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Body_Mass""",false,179,"""179 rows with out of bound val…","""value_error.rowvalue"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""
"""Sex""",false,0,"""Rows with invalid values: {Non…","""value_error.rowvalue"""


In [45]:
# 確認 Body_Mass 的值是否皆為 5 的倍數

class P08_1(pt.Model):
    ID: int = pt.Field(unique=True)
    Species: Literal['Gentoo penguin (Pygoscelis papua)', 
                     'Adelie Penguin (Pygoscelis adeliae)', 
                     'Chinstrap penguin (Pygoscelis antarctica)']
    Island: Literal['Torgersen', 'Biscoe', 'Dream']
    Clutch_Completion: Literal['Yes', 'No']
    Culmen_Length: float
    Culmen_Depth: float
    Flipper_Length: int
    Body_Mass: int = pt.Field(multiple_of=5)
    Sex: Literal['MALE', 'FEMALE']
    Comments: Optional[str]
    Island_Code: int

In [46]:
val(P08_1, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",false,316,"""316 rows with duplicated value…","""value_error.rowvalue"""
"""Species""",true,0,null,null
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Body_Mass""",false,2,"""2 missing values""","""value_error.missingvalues"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""
"""Sex""",false,0,"""Rows with invalid values: {Non…","""value_error.rowvalue"""


# 9. 值為常數

確認該欄位的值皆為常數（所有的值皆相同）。

In [47]:
df.filter(pl.col('Body_Mass')!=5000).shape

(336, 11)

In [48]:
# 確認 Body_Mass 的值是否皆為 5000

class P09(pt.Model):
    ID: int = pt.Field(unique=True)
    Species: Literal['Gentoo penguin (Pygoscelis papua)', 
                     'Adelie Penguin (Pygoscelis adeliae)', 
                     'Chinstrap penguin (Pygoscelis antarctica)']
    Island: Literal['Torgersen', 'Biscoe', 'Dream']
    Clutch_Completion: Literal['Yes', 'No']
    Culmen_Length: float
    Culmen_Depth: float
    Flipper_Length: int
    Body_Mass: Literal[5000]
    Sex: Literal['MALE', 'FEMALE']
    Comments: Optional[str]
    Island_Code: int

In [49]:
val(P09, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",false,316,"""316 rows with duplicated value…","""value_error.rowvalue"""
"""Species""",true,0,null,null
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Body_Mass""",false,336,"""336 rows with out of bound val…","""value_error.rowvalue"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""
"""Sex""",false,0,"""Rows with invalid values: {Non…","""value_error.rowvalue"""


# 10. 某個欄位加總為特定值

確認該欄位的加總為特定值，常見於欄位的所有值加總必須等於 1（比例資料）。

In [50]:
df.select(pl.col('Body_Mass').sum())

Body_Mass
i64
1437000


In [51]:
# 確認 Body_Mass 的值是否總和為 1437000
# pt.field 等價於 pl.col('Body_Mass')

class P10(pt.Model):
    ID: int = pt.Field(unique=True)
    Species: Literal['Gentoo penguin (Pygoscelis papua)', 
                     'Adelie Penguin (Pygoscelis adeliae)', 
                     'Chinstrap penguin (Pygoscelis antarctica)']
    Island: Literal['Torgersen', 'Biscoe', 'Dream']
    Clutch_Completion: Literal['Yes', 'No']
    Culmen_Length: float
    Culmen_Depth: float
    Flipper_Length: int
    Body_Mass: int = pt.Field(constraints=pt.field.sum() == 1437000)
    Sex: Literal['MALE', 'FEMALE']
    Comments: Optional[str]
    Island_Code: int

In [52]:
val(P10, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",false,316,"""316 rows with duplicated value…","""value_error.rowvalue"""
"""Species""",true,0,null,null
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Body_Mass""",false,2,"""2 missing values""","""value_error.missingvalues"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""
"""Sex""",false,0,"""Rows with invalid values: {Non…","""value_error.rowvalue"""


In [53]:
class P10_1(pt.Model):
    ID: int = pt.Field(unique=True)
    Species: Literal['Gentoo penguin (Pygoscelis papua)', 
                     'Adelie Penguin (Pygoscelis adeliae)', 
                     'Chinstrap penguin (Pygoscelis antarctica)']
    Island: Literal['Torgersen', 'Biscoe', 'Dream']
    Clutch_Completion: Literal['Yes', 'No']
    Culmen_Length: float
    Culmen_Depth: float
    Flipper_Length: int
    Body_Mass: int = pt.Field(constraints=pt.field.sum() == 1400000)
    Sex: Literal['MALE', 'FEMALE']
    Comments: Optional[str]
    Island_Code: int

In [54]:
val(P10_1, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",false,316,"""316 rows with duplicated value…","""value_error.rowvalue"""
"""Species""",true,0,null,null
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Body_Mass""",false,344,"""344 rows does not match custom…","""value_error.rowvalue"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""
"""Sex""",false,0,"""Rows with invalid values: {Non…","""value_error.rowvalue"""


# 11. 欄位之間的關係

確認不同欄位之間的關係是否成立，例如某欄位的值必須大於另一欄位。

In [55]:
class P11(pt.Model):
    ID: int = pt.Field(unique=True)
    Species: Literal['Gentoo penguin (Pygoscelis papua)', 
                     'Adelie Penguin (Pygoscelis adeliae)', 
                     'Chinstrap penguin (Pygoscelis antarctica)']
    Island: Literal['Torgersen', 'Biscoe', 'Dream']
    Clutch_Completion: Literal['Yes', 'No']
    Culmen_Length: float = pt.Field(constraints=pl.col('Culmen_Length') > pl.col('Culmen_Depth'))
    Culmen_Depth: float
    Flipper_Length: int
    Body_Mass: int
    Sex: Literal['MALE', 'FEMALE']
    Comments: Optional[str]
    Island_Code: int

In [56]:
val(P11, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",false,316,"""316 rows with duplicated value…","""value_error.rowvalue"""
"""Species""",true,0,null,null
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Body_Mass""",false,2,"""2 missing values""","""value_error.missingvalues"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""
"""Sex""",false,0,"""Rows with invalid values: {Non…","""value_error.rowvalue"""


In [57]:
df.filter(~(pl.col('Culmen_Length') > pl.col('Culmen_Depth'))).select(pl.all())

ID,Species,Island,Clutch_Completion,Culmen_Length,Culmen_Depth,Flipper_Length,Body_Mass,Sex,Comments,Island_Code
i64,str,str,str,f64,f64,i64,i64,str,str,i64
11,"""Adelie Penguin (Pygoscelis ade…","""Torgersen""","""Yes""",4.1,17.1,186,3300,null,"""No blood sample obtained for s…",0


# 12. 條件關係

更進階的欄位關係檢查。根據欄位值的不同，檢查不同的條件是否成立。

In [58]:
# 檢查當 Island 為 Torgersen 時，Island_Code 是否為 0
# 檢查當 Island 為 Biscoe 時，Island_Code 是否為 1
# 檢查當 Island 為 Dream 時，Island_Code 是否為 2

class P12(pt.Model):
    ID: int = pt.Field(unique=True)
    Species: Literal['Gentoo penguin (Pygoscelis papua)', 
                     'Adelie Penguin (Pygoscelis adeliae)', 
                     'Chinstrap penguin (Pygoscelis antarctica)']
    Island: Literal['Torgersen', 'Biscoe', 'Dream']
    Clutch_Completion: Literal['Yes', 'No']
    Culmen_Length: float = pt.Field(constraints=pl.col('Culmen_Length') > pl.col('Culmen_Depth'))
    Culmen_Depth: float
    Flipper_Length: int
    Body_Mass: int
    Sex: Literal['MALE', 'FEMALE']
    Comments: Optional[str]
    Island_Code: int = pt.Field(constraints=pl.when(pl.col('Island')=='Torgersen')
                                             .then(pt.field == 0)
                                             .when(pl.col('Island')=='Biscoe')
                                             .then(pt.field == 1).otherwise(pt.field == 2))

In [59]:
val(P12, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",false,316,"""316 rows with duplicated value…","""value_error.rowvalue"""
"""Species""",true,0,null,null
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Body_Mass""",false,2,"""2 missing values""","""value_error.missingvalues"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""
"""Sex""",false,0,"""Rows with invalid values: {Non…","""value_error.rowvalue"""


In [60]:
df.filter(~(pl.when(pl.col('Island')=='Torgersen')
            .then(pl.col('Island_Code') == 0)
            .when(pl.col('Island')=='Biscoe')
            .then(pl.col('Island_Code') == 1)
            .otherwise(pl.col('Island_Code') == 2)))\
    .select(pl.all())

ID,Species,Island,Clutch_Completion,Culmen_Length,Culmen_Depth,Flipper_Length,Body_Mass,Sex,Comments,Island_Code
i64,str,str,str,f64,f64,i64,i64,str,str,i64
29,"""Adelie Penguin (Pygoscelis ade…","""Biscoe""","""No""",37.9,18.6,172,3150,"""FEMALE""","""Nest never observed with full …",0


# 13. 多重關係

檢查一欄位內的多種條件是否都成立。

In [61]:
# 檢查 Culmen_Length 是否大於 Culmen_Depth
# 檢查 Culmen_Length 是否小於 Body_Mass

class P13(pt.Model):
    ID: int = pt.Field(unique=True)
    Species: Literal['Gentoo penguin (Pygoscelis papua)', 
                     'Adelie Penguin (Pygoscelis adeliae)', 
                     'Chinstrap penguin (Pygoscelis antarctica)']
    Island: Literal['Torgersen', 'Biscoe', 'Dream']
    Clutch_Completion: Literal['Yes', 'No']
    Culmen_Length: float = pt.Field(constraints=[pl.col('Culmen_Length') > pl.col('Culmen_Depth'),
                                                 pl.col('Body_Mass') > pl.col('Culmen_Length')])
    Culmen_Depth: float
    Flipper_Length: int
    Body_Mass: int
    Sex: Literal['MALE', 'FEMALE']
    Comments: Optional[str]
    Island_Code: int

In [62]:
val(P13, df)

column,pass,failed_count,msg,type
str,bool,i64,str,str
"""ID""",false,316,"""316 rows with duplicated value…","""value_error.rowvalue"""
"""Species""",true,0,null,null
"""Island""",true,0,null,null
"""Clutch_Completion""",true,0,null,null
"""Culmen_Length""",false,2,"""2 missing values""","""value_error.missingvalues"""
…,…,…,…,…
"""Body_Mass""",false,2,"""2 missing values""","""value_error.missingvalues"""
"""Sex""",false,10,"""10 missing values""","""value_error.missingvalues"""
"""Sex""",false,0,"""Rows with invalid values: {Non…","""value_error.rowvalue"""


In [63]:
df.filter(~((pl.col('Culmen_Length') > pl.col('Culmen_Depth')) & (pl.col('Body_Mass') > pl.col('Culmen_Length'))))\
    .select(pl.all())

ID,Species,Island,Clutch_Completion,Culmen_Length,Culmen_Depth,Flipper_Length,Body_Mass,Sex,Comments,Island_Code
i64,str,str,str,f64,f64,i64,i64,str,str,i64
11,"""Adelie Penguin (Pygoscelis ade…","""Torgersen""","""Yes""",4.1,17.1,186,3300,null,"""No blood sample obtained for s…",0
108,"""Adelie Penguin (Pygoscelis ade…","""Biscoe""","""Yes""",3910.0,20.0,190,3900,"""MALE""",null,1
105,"""Gentoo penguin (Pygoscelis pap…","""Biscoe""","""Yes""",4730.0,13.8,216,4725,null,null,1


In [64]:
df.group_by('Island', 'Species').count()

/var/folders/zz/x65716_j4s7947htd3809l4h0000gn/T/ipykernel_91424/874737968.py:1: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  df.group_by('Island', 'Species').count()


Island,Species,count
str,str,u32
"""Biscoe""","""Gentoo penguin (Pygoscelis pap…",124
"""Dream""","""Chinstrap penguin (Pygoscelis …",68
"""Torgersen""","""Adelie Penguin (Pygoscelis ade…",52
"""Biscoe""","""Adelie Penguin (Pygoscelis ade…",44
"""Dream""","""Adelie Penguin (Pygoscelis ade…",56


藉由以上的 14 種常見的資料驗證情境，相信各位能體會 `patito` 搭配 `polars` 的強大之處了！

接下來，請各位應用剛剛所學的內容，設計更完整的資料驗證規則吧！

可以添加的規則：
* 欄位是否需大於 0
* 特定的企鵝是否只會在某些島上發現

實務上，如何設計好的資料驗證規則非常仰賴領域知識，因此若對於資料集不熟悉的話，建議可以找該領域的專家進行詢問喔！

如果想要學習更多的話，可以到 [patito 官網](https://patito.readthedocs.io/en/latest/) 學習更多指令的用法，或者到 [polars 官網](https://docs.pola.rs/api/python/stable/reference/index.html) 學習更進階的語法，協助你設計出更完整的資料驗證規則 :)